In [139]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import os
%pwd

u'/Users/Driz/DSI-CHI-1'

In [3]:
data = pd.read_csv('../data_sets/job_data.csv')

In [4]:
# looks like I might need to reset index to fix whatever the 'Unnamed' column is doing
data.head()

,Unnamed: 0,Company Name,High Salary,Job Title,Location,Salary,Summary
0,0,Digital Factory,0,Chief Data Scientist (equity position),"Chicago, IL",55000,Chief Data Scientist*. We are looking for an e...
1,1,Anomalix,0,Data Scientist,"Chicago, IL",55000,Data Scientist Responsibilities will include:....
2,2,GE Transportation,0,Senior Data Scientist,"Chicago, IL",55000,The Senior Data Scientist will demonstrate lea...
3,3,Google,0,"Machine Learning Deployment Engineer, Professi...","Chicago, IL",55000,Experience interpreting technical data and cre...
4,4,Civis Analytics,0,Data Scientist - Unstructured Data,"Chicago, IL",55000,"With a specialization in unstructured data, th..."


In [175]:
data.describe()
data['new_high_salary'] = data['Salary'].apply(lambda x: 1 if x > 100000 else 0)
data['new_high_salary'].describe()

count    1650.000000
mean        0.400000
std         0.490046
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: new_high_salary, dtype: float64

In [173]:
data.pivot_table(index='High Salary', values='Salary', aggfunc=np.max)

High Salary
0     80000
1    120000
Name: Salary, dtype: int64

In [7]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
data.head()

,Company Name,High Salary,Job Title,Location,Salary,Summary
0,Digital Factory,0,Chief Data Scientist (equity position),"Chicago, IL",55000,Chief Data Scientist*. We are looking for an e...
1,Anomalix,0,Data Scientist,"Chicago, IL",55000,Data Scientist Responsibilities will include:....
2,GE Transportation,0,Senior Data Scientist,"Chicago, IL",55000,The Senior Data Scientist will demonstrate lea...
3,Google,0,"Machine Learning Deployment Engineer, Professi...","Chicago, IL",55000,Experience interpreting technical data and cre...
4,Civis Analytics,0,Data Scientist - Unstructured Data,"Chicago, IL",55000,"With a specialization in unstructured data, th..."


In [26]:
data.columns

Index([u'Company Name', u'High Salary', u'Job Title', u'Location', u'Salary',
       u'Summary'],
      dtype='object')

In [27]:
data.Location.unique()

array(['Chicago, IL', 'Chicago, IL 60604 (Loop area)',
       'Chicago, IL 60611 (Near North Side area)',
       'Chicago, IL 60654 (Loop area)', 'Chicago, IL 60601 (Loop area)',
       'Chicago, IL 60606 (Loop area)', 'Deerfield, IL',
       'Chicago, IL 60603 (Loop area)',
       'Chicago, IL 60660 (Edgewater area)', 'Northbrook, IL',
       'Chicago Heights, IL', 'Northbrook, IL 60062', 'Des Plaines, IL',
       'Deerfield, IL 60015', 'Rolling Meadows, IL', 'Lemont, IL',
       'Chicago, IL 60642 (Portage Park area)'], dtype=object)

In [28]:
# rename
cols = ['Chicago, IL', 'Chicago, IL 60604',
       'Chicago, IL 60611',
       'Chicago, IL 60654', 'Chicago, IL 60601',
       'Chicago, IL 60606', 'Deerfield, IL',
       'Chicago, IL 60603',
       'Chicago, IL 60660', 'Northbrook, IL',
       'Chicago Heights, IL', 'Northbrook, IL', 'Des Plaines, IL',
       'Deerfield, IL', 'Rolling Meadows, IL', 'Lemont, IL',
       'Chicago, IL 60642']

In [30]:
df_locations.columns = cols

ValueError: Length mismatch: Expected axis has 17 elements, new values have 16 elements

In [ ]:
# now I need to think about dummie_variables and key words

In [18]:
df_locations = pd.get_dummies(data['Location'])

In [20]:
df_locations.head()

,"Chicago Heights, IL","Chicago, IL","Chicago, IL 60601 (Loop area)","Chicago, IL 60603 (Loop area)","Chicago, IL 60604 (Loop area)","Chicago, IL 60606 (Loop area)","Chicago, IL 60611 (Near North Side area)","Chicago, IL 60642 (Portage Park area)","Chicago, IL 60654 (Loop area)","Chicago, IL 60660 (Edgewater area)","Deerfield, IL","Deerfield, IL 60015","Des Plaines, IL","Lemont, IL","Northbrook, IL","Northbrook, IL 60062","Rolling Meadows, IL"
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# rename cause the column titles are messy
cols = ['Chicago, IL', 'Chicago, IL 60604',
       'Chicago, IL 60611',
       'Chicago, IL 60654', 'Chicago, IL 60601',
       'Chicago, IL 60606', 'Deerfield, IL',
       'Chicago, IL 60603',
       'Chicago, IL 60660', 'Northbrook, IL',
       'Chicago Heights, IL', 'Northbrook, IL', 'Des Plaines, IL',
       'Deerfield, IL', 'Rolling Meadows, IL', 'Lemont, IL',
       'Chicago, IL 60642']

In [34]:
df_locations.columns = cols

In [35]:
df_locations.head()

,"Chicago, IL","Chicago, IL 60604","Chicago, IL 60611","Chicago, IL 60654","Chicago, IL 60601","Chicago, IL 60606","Deerfield, IL","Chicago, IL 60603","Chicago, IL 60660","Northbrook, IL","Chicago Heights, IL","Northbrook, IL","Des Plaines, IL","Deerfield, IL","Rolling Meadows, IL","Lemont, IL","Chicago, IL 60642"
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Here I want to pull common words from job title and job description
Possibly put into separate tables, not sure yet

In [70]:
vector_data = CountVectorizer(
    binary=True, # creates binary features (1,0)
    stop_words='english', #ignores common words
    max_features=20) #grabs top 50 common words

In [74]:
# builds matrix, row per website/column per word (using all filtered words)
summary_table = vector_data.fit_transform(data['Summary']).todense()
summary_table = pd.DataFrame(vector_matrix, columns=vector_data.get_feature_names())
summary_table.head()

,build,data,develop,engineers,experience,manage,management,mentor,mining,modeling,programs,rwi,scientist,scientists,services,smart,support,team,using,working
0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [75]:
summary_table.describe()

,build,data,develop,engineers,experience,manage,management,mentor,mining,modeling,programs,rwi,scientist,scientists,services,smart,support,team,using,working
count,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.00000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000
mean,0.272727,0.718182,0.151515,0.145455,0.290909,0.139394,0.151515,0.139394,0.148485,0.145455,0.139394,0.139394,0.19697,0.442424,0.139394,0.260606,0.139394,0.166667,0.154545,0.145455
std,0.445497,0.450022,0.358659,0.352665,0.454319,0.346462,0.358659,0.346462,0.355688,0.352665,0.346462,0.346462,0.39783,0.496825,0.346462,0.439098,0.346462,0.372791,0.361580,0.352665
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [76]:
# after this step go ahead and begin logistic regression
summary_table.sum().sort_values(ascending=False)

data          1185
scientists     730
experience     480
build          450
smart          430
scientist      325
team           275
using          255
management     250
develop        250
mining         245
engineers      240
working        240
modeling       240
mentor         230
manage         230
programs       230
rwi            230
services       230
support        230
dtype: int64

I also ran count vectorize max features through the entire data frame. I wanted to see if some key words that do not appear often may be a contributing factor in high salary. There was quite a bit to navigate through. Given more time I would probably go through them and pluck out interesting looking key words. I also noticed that it did not return the results I was hoping for. I wanted to see keywords such as: Spark, Hadoop, SQL, Python, Tableau, Machine Learning, Modeling, Predictive Learning, etc. I feel like these key words would be more fruitful in exploring the relationship between job postings and high salary. One reason using those key words was not possible in this project is that most of those terms were found when clicking through the job posting which brings you to a different web page (not always hosted by Indeed), which at this point, is beyond my scope of web scraping abilities.  

In [ ]:
# vector on job title now and make a separate table

In [ ]:
vector_data = CountVectorizer(
    binary=True, # creates binary features (1,0)
    stop_words='english', #ignores common words
    max_features=20) #grabs top 50 common words

In [78]:
# builds matrix, row per website/column per word (using all filtered words)
job_title_table = vector_data.fit_transform(data['Job Title']).todense()
job_title_table = pd.DataFrame(job_title_table, columns=vector_data.get_feature_names())
job_title_table.head()

,chief,clinical,credit,data,developer,development,direc,director,end,engineer,exploratory,officer,pharmacology,rails,ruby,rwi,science,scientist,senior,software
0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [79]:
# after this step go ahead and begin logistic regression
job_title_table.sum().sort_values(ascending=False)

senior          510
science         460
data            430
developer       430
engineer        270
director        265
software        230
rwi             225
clinical        220
chief           220
pharmacology    215
officer         215
exploratory     215
rails           215
end             215
ruby            215
direc           215
development     215
credit          215
scientist       145
dtype: int64

Ok, so now I have four tables:
    data : original dataframe with all my variables
    summary_table : job summary count vectorized
    job_title_table : job title count vectorized
    df_locations : location variable with dummie_vars
    
Now I want to run logistic regressions on the last three tables and then possibly join all three into a new df and run regressions on that as well. 

Here we go!

In [81]:
summary_table.shape

(1650, 20)

In [82]:
job_title_table.shape

(1650, 20)

In [83]:
df_locations.shape

(1650, 17)

In [97]:
job_hunt = pd.DataFrame(result)
job_hunt.head()

,build,data,develop,engineers,experience,manage,management,mentor,mining,modeling,...,"Chicago, IL 60660","Northbrook, IL","Chicago Heights, IL","Northbrook, IL","Des Plaines, IL","Deerfield, IL","Rolling Meadows, IL","Lemont, IL","Chicago, IL 60642",High Salary
0,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


Going to re-do this with a join instead to prevent duplicate columns

In [98]:
job_hunt.columns

Index([              u'build',                u'data',             u'develop',
                 u'engineers',          u'experience',              u'manage',
                u'management',              u'mentor',              u'mining',
                  u'modeling',            u'programs',                 u'rwi',
                 u'scientist',          u'scientists',            u'services',
                     u'smart',             u'support',                u'team',
                     u'using',             u'working',               u'chief',
                  u'clinical',              u'credit',                u'data',
                 u'developer',         u'development',               u'direc',
                  u'director',                 u'end',            u'engineer',
               u'exploratory',             u'officer',        u'pharmacology',
                     u'rails',                u'ruby',                 u'rwi',
                   u'science',           u'scientist

In [99]:
new_table = summary_table.join(job_title_table, lsuffix='summary', rsuffix='job_table')

In [100]:
new_table.head()

,build,datasummary,develop,engineers,experience,manage,management,mentor,mining,modeling,...,exploratory,officer,pharmacology,rails,ruby,rwijob_table,science,scientistjob_table,senior,software
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [101]:
modeling_table = new_table.join(df_locations, lsuffix='location')

In [102]:
modeling_table.head()

,build,datasummary,develop,engineers,experience,manage,management,mentor,mining,modeling,...,"Chicago, IL 60603","Chicago, IL 60660","Northbrook, IL","Chicago Heights, IL","Northbrook, IL","Des Plaines, IL","Deerfield, IL","Rolling Meadows, IL","Lemont, IL","Chicago, IL 60642"
0,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


My modeling_table above contains all my variable columns that I want to work my logistic regressions off of. Key terms come from the summary unless otherwise listed in column heading. 

In [103]:
# this is not easy identify due to large number of variables, look for better technique
modeling_table.corr()

,build,datasummary,develop,engineers,experience,manage,management,mentor,mining,modeling,...,"Chicago, IL 60603","Chicago, IL 60660","Northbrook, IL","Chicago Heights, IL","Northbrook, IL","Des Plaines, IL","Deerfield, IL","Rolling Meadows, IL","Lemont, IL","Chicago, IL 60642"
build,1.000000,-0.266735,-0.258775,-0.214047,0.266936,-0.246454,-0.258775,-0.246454,-0.255718,-0.233346,...,0.632088,-0.058655,-0.033761,-0.047818,-0.033761,-0.033761,-0.237033,-0.369274,-0.033761,-0.047818
datasummary,-0.266735,1.000000,0.264712,0.239337,0.386401,0.252108,-0.580658,0.252108,0.261585,0.239337,...,-0.617911,-0.010968,0.034536,0.048915,0.034536,0.034536,0.242471,-0.277218,0.034536,0.048915
develop,-0.258775,0.264712,1.000000,-0.126398,-0.270666,0.927986,-0.178571,0.927986,0.893113,0.904399,...,-0.163568,0.048571,-0.023297,-0.032998,-0.023297,-0.023297,-0.163568,0.586094,0.130466,0.184787
engineers,-0.214047,0.239337,-0.126398,1.000000,-0.245331,-0.166041,-0.174342,-0.166041,-0.172282,-0.145833,...,-0.159694,0.051043,-0.022746,-0.032216,-0.022746,-0.022746,-0.159694,-0.248788,-0.022746,0.189270
experience,0.266936,0.386401,-0.270666,-0.245331,1.000000,-0.257779,-0.252058,-0.257779,-0.267468,-0.245331,...,-0.247925,-0.061350,0.086074,-0.050016,0.086074,-0.035313,0.604318,-0.371156,-0.035313,-0.050016
manage,-0.246454,0.252108,0.927986,-0.166041,-0.257779,1.000000,-0.170069,1.000000,0.939168,0.950676,...,-0.155780,-0.038548,-0.022188,-0.031427,-0.022188,-0.022188,-0.155780,0.627831,0.136988,-0.031427
management,-0.258775,-0.580658,-0.178571,-0.174342,-0.252058,-0.170069,1.000000,-0.170069,-0.176462,-0.126398,...,-0.163568,-0.040476,-0.023297,0.075895,0.130466,-0.023297,-0.163568,0.566983,-0.023297,-0.032998
mentor,-0.246454,0.252108,0.927986,-0.166041,-0.257779,1.000000,-0.170069,1.000000,0.939168,0.950676,...,-0.155780,-0.038548,-0.022188,-0.031427,-0.022188,-0.022188,-0.155780,0.627831,0.136988,-0.031427
mining,-0.255718,0.261585,0.893113,-0.172282,-0.267468,0.939168,-0.176462,0.939168,1.000000,0.915470,...,-0.161636,-0.039997,-0.023022,-0.032608,-0.023022,-0.023022,-0.161636,0.596129,0.132025,-0.032608
modeling,-0.233346,0.239337,0.904399,-0.145833,-0.245331,0.950676,-0.126398,0.950676,0.915470,1.000000,...,-0.159694,-0.039517,-0.022746,-0.032216,-0.022746,-0.022746,-0.159694,0.606421,0.133631,-0.032216


The correlation table above may show some hints as to which variables I want to test. The key words; develop, mentor, modeling, and mining seem to yield strong correlation scores. Will keep those key words in mind moving forward.

Let's build a plain ol' logistic regression and get going already!

In [ ]:
# target will be 'high_salary' will need to pull it from data['High Salary'] because it is not
# attached to my modeling_table df

In [113]:
# trash because it has ALL the variables
trash_model = LogisticRegression()

In [109]:
# uses all variables
everything = modeling_table.columns

In [176]:
y = data['High Salary']
X = modeling_table[everything]

In [177]:
data['High Salary'].value_counts()

0    990
1    660
Name: High Salary, dtype: int64

In [178]:
# can go back to change C and L2 
trash_model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [179]:
trash_model.score(X, y)

0.59999999999999998

In [180]:
predictions = trash_model.predict(X)
predictions[0:]

array([0, 0, 0, ..., 0, 0, 0])

In [181]:
predict_proba = trash_model.predict_proba(X)
predict_proba[0:]

array([[ 0.59976053,  0.40023947],
       [ 0.60116555,  0.39883445],
       [ 0.60189321,  0.39810679],
       ..., 
       [ 0.59993819,  0.40006181],
       [ 0.60002444,  0.39997556],
       [ 0.59994915,  0.40005085]])

In [182]:
# Confusion Matrix!
y_pred = trash_model.predict(X)
confusion = np.array(confusion_matrix(y, y_pred))
print confusion

[[990   0]
 [660   0]]


In [183]:
print classification_report(y, y_pred)

             precision    recall  f1-score   support

          0       0.60      1.00      0.75       990
          1       0.00      0.00      0.00       660

avg / total       0.36      0.60      0.45      1650



Well that did not go smoothly. Let's try less X variables 

In [184]:
modeling_table.columns

Index([              u'build',         u'datasummary',             u'develop',
                 u'engineers',          u'experience',              u'manage',
                u'management',              u'mentor',              u'mining',
                  u'modeling',            u'programs',          u'rwisummary',
          u'scientistsummary',          u'scientists',            u'services',
                     u'smart',             u'support',                u'team',
                     u'using',             u'working',               u'chief',
                  u'clinical',              u'credit',       u'datajob_table',
                 u'developer',         u'development',               u'direc',
                  u'director',                 u'end',            u'engineer',
               u'exploratory',             u'officer',        u'pharmacology',
                     u'rails',                u'ruby',        u'rwijob_table',
                   u'science',  u'scientistjob_table

In [185]:
model_2 = LogisticRegression()

In [186]:
y = data['High Salary']
X = modeling_table[['scientistjob_table', 'rails', 'ruby',
                   'developer', 'director', 'modeling',
                   'mining', 'chief', 'engineer', 
                   'Chicago, IL','Chicago, IL 60604',
                   'Chicago, IL 60611', 'Chicago, IL 60654',
                   'Chicago, IL 60601', 'Chicago, IL 60606',       
                   'Deerfield, IL', 'Chicago, IL 60603',
                   'Chicago, IL 60660', 'Northbrook, IL', 
                   'Chicago Heights, IL', 'Northbrook, IL',     
                   'Des Plaines, IL', 'Deerfield, IL',
                   'Rolling Meadows, IL', 'Lemont, IL', 'Chicago, IL 60642']]

In [187]:
model_2.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [188]:
model_2.score(X,y)

0.59999999999999998

In [189]:
y_pred = model_2.predict(X)
confusion = np.array(confusion_matrix(y, y_pred))
print confusion

[[990   0]
 [660   0]]


In [203]:
model_3 = LogisticRegression(penalty='l1', C=0.01)

In [204]:
y = data['High Salary']
X = modeling_table[['scientistjob_table', 'rails', 'ruby',
                   'developer', 'director', 'modeling',
                   'mining', 'chief', 'engineer']]

In [205]:
model_3.fit(X,y)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [206]:
model_3.score(X,y)

0.59999999999999998

In [207]:
X.head()

,scientistjob_table,rails,ruby,developer,director,modeling,mining,chief,engineer
0,1,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0


In [195]:
X['scientistjob_table'].describe()

count    1650.000000
mean        0.087879
std         0.283204
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: scientistjob_table, dtype: float64

In [196]:
model_single = LogisticRegression()

In [197]:
X = modeling_table[['chief', 'director']]
y = data['High Salary']

In [198]:
model_single.fit(X ,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [199]:
model_single.score(X,y)

0.59999999999999998

In [200]:
data['High Salary'].describe()

count    1650.000000
mean        0.400000
std         0.490046
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: High Salary, dtype: float64

In [201]:
corr_df = pd.DataFrame(data['High Salary']).join(modeling_table)

In [202]:
corr_df.corr()

,High Salary,build,datasummary,develop,engineers,experience,manage,management,mentor,mining,...,"Chicago, IL 60603","Chicago, IL 60660","Northbrook, IL","Chicago Heights, IL","Northbrook, IL","Des Plaines, IL","Deerfield, IL","Rolling Meadows, IL","Lemont, IL","Chicago, IL 60642"
High Salary,1.000000e+00,-2.379270e-16,8.227686e-17,-5.966211e-17,-9.378983e-17,-1.059935e-16,-1.338848e-16,-6.090707e-17,-1.338848e-16,4.504795e-17,...,2.064477e-16,3.308670e-16,-3.200649e-17,-4.887223e-17,-3.200649e-17,-3.200649e-17,1.964517e-16,-3.459958e-16,-2.950766e-17,-4.887223e-17
build,-2.379270e-16,1.000000e+00,-2.667353e-01,-2.587746e-01,-2.140469e-01,2.669359e-01,-2.464537e-01,-2.587746e-01,-2.464537e-01,-2.557175e-01,...,6.320881e-01,-5.865464e-02,-3.376119e-02,-4.781825e-02,-3.376119e-02,-3.376119e-02,-2.370331e-01,-3.692745e-01,-3.376119e-02,-4.781825e-02
datasummary,8.227686e-17,-2.667353e-01,1.000000e+00,2.647118e-01,2.393369e-01,3.864010e-01,2.521082e-01,-5.806581e-01,2.521082e-01,2.615846e-01,...,-6.179111e-01,-1.096781e-02,3.453579e-02,4.891537e-02,3.453579e-02,3.453579e-02,2.424714e-01,-2.772175e-01,3.453579e-02,4.891537e-02
develop,-5.966211e-17,-2.587746e-01,2.647118e-01,1.000000e+00,-1.263980e-01,-2.706660e-01,9.279864e-01,-1.785714e-01,9.279864e-01,8.931131e-01,...,-1.635683e-01,4.857066e-02,-2.329743e-02,-3.299773e-02,-2.329743e-02,-2.329743e-02,-1.635683e-01,5.860943e-01,1.304656e-01,1.847873e-01
engineers,-9.378983e-17,-2.140469e-01,2.393369e-01,-1.263980e-01,1.000000e+00,-2.453308e-01,-1.660412e-01,-1.743420e-01,-1.660412e-01,-1.722824e-01,...,-1.596943e-01,5.104266e-02,-2.274564e-02,-3.221619e-02,-2.274564e-02,-2.274564e-02,-1.596943e-01,-2.487882e-01,-2.274564e-02,1.892701e-01
experience,-1.059935e-16,2.669359e-01,3.864010e-01,-2.706660e-01,-2.453308e-01,1.000000e+00,-2.577789e-01,-2.520577e-01,-2.577789e-01,-2.674685e-01,...,-2.479254e-01,-6.134998e-02,8.607447e-02,-5.001563e-02,8.607447e-02,-3.531260e-02,6.043181e-01,-3.711560e-01,-3.531260e-02,-5.001563e-02
manage,-1.338848e-16,-2.464537e-01,2.521082e-01,9.279864e-01,-1.660412e-01,-2.577789e-01,1.000000e+00,-1.700692e-01,1.000000e+00,9.391682e-01,...,-1.557805e-01,-3.854841e-02,-2.218819e-02,-3.142663e-02,-2.218819e-02,-2.218819e-02,-1.557805e-01,6.278307e-01,1.369879e-01,-3.142663e-02
management,-6.090707e-17,-2.587746e-01,-5.806581e-01,-1.785714e-01,-1.743420e-01,-2.520577e-01,-1.700692e-01,1.000000e+00,-1.700692e-01,-1.764619e-01,...,-1.635683e-01,-4.047555e-02,-2.329743e-02,7.589478e-02,1.304656e-01,-2.329743e-02,-1.635683e-01,5.669825e-01,-2.329743e-02,-3.299773e-02
mentor,-1.338848e-16,-2.464537e-01,2.521082e-01,9.279864e-01,-1.660412e-01,-2.577789e-01,1.000000e+00,-1.700692e-01,1.000000e+00,9.391682e-01,...,-1.557805e-01,-3.854841e-02,-2.218819e-02,-3.142663e-02,-2.218819e-02,-2.218819e-02,-1.557805e-01,6.278307e-01,1.369879e-01,-3.142663e-02
mining,4.504795e-17,-2.557175e-01,2.615846e-01,8.931131e-01,-1.722824e-01,-2.674685e-01,9.391682e-01,-1.764619e-01,9.391682e-01,1.000000e+00,...,-1.616360e-01,-3.999739e-02,-2.302221e-02,-3.260791e-02,-2.302221e-02,-2.302221e-02,-1.616360e-01,5.961293e-01,1.320253e-01,-3.260791e-02


In [171]:
for column in job_hunt.columns:
    print column

build
data
develop
engineers
experience
manage
management
mentor
mining
modeling
programs
rwi
scientist
scientists
services
smart
support
team
using
working
chief
clinical
credit
data
developer
development
direc
director
end
engineer
exploratory
officer
pharmacology
rails
ruby
rwi
science
scientist
senior
software
Chicago, IL
Chicago, IL 60604
Chicago, IL 60611
Chicago, IL 60654
Chicago, IL 60601
Chicago, IL 60606
Deerfield, IL
Chicago, IL 60603
Chicago, IL 60660
Northbrook, IL
Chicago Heights, IL
Northbrook, IL
Des Plaines, IL
Deerfield, IL
Rolling Meadows, IL
Lemont, IL
Chicago, IL 60642
High Salary


In [174]:
print corr_df[['High Salary', 'scientist']]

KeyError: "['scientist'] not in index"

Just playing around

In [208]:
data.head()

,Company Name,High Salary,Job Title,Location,Salary,Summary,new_high_salary
0,Digital Factory,0,Chief Data Scientist (equity position),"Chicago, IL",55000,Chief Data Scientist*. We are looking for an e...,0
1,Anomalix,0,Data Scientist,"Chicago, IL",55000,Data Scientist Responsibilities will include:....,0
2,GE Transportation,0,Senior Data Scientist,"Chicago, IL",55000,The Senior Data Scientist will demonstrate lea...,0
3,Google,0,"Machine Learning Deployment Engineer, Professi...","Chicago, IL",55000,Experience interpreting technical data and cre...,0
4,Civis Analytics,0,Data Scientist - Unstructured Data,"Chicago, IL",55000,"With a specialization in unstructured data, th...",0


In [210]:
data['Location'].value_counts()

Chicago, IL                                 670
Northbrook, IL                              440
Lemont, IL                                  215
Chicago, IL 60642 (Portage Park area)       215
Chicago, IL 60654 (Loop area)                15
Chicago, IL 60603 (Loop area)                15
Rolling Meadows, IL                          10
Chicago, IL 60606 (Loop area)                10
Chicago, IL 60601 (Loop area)                10
Chicago, IL 60611 (Near North Side area)     10
Deerfield, IL                                10
Northbrook, IL 60062                          5
Chicago, IL 60660 (Edgewater area)            5
Chicago, IL 60604 (Loop area)                 5
Chicago Heights, IL                           5
Des Plaines, IL                               5
Deerfield, IL 60015                           5
Name: Location, dtype: int64

In [214]:
data[['Company Name', 'Job Title', 'Location', 'Summary']].duplicated().sum()

1572

In [216]:
data['unique_id'] = data['Company Name'] + data['Job Title'] + data['Location'] + data['Summary']
data['unique_id'].duplicated().sum()

1572

In [218]:
data = data.sort_values(by='Salary', ascending=True)
data.head()

,Company Name,High Salary,Job Title,Location,Salary,Summary,new_high_salary,unique_id
0,Digital Factory,0,Chief Data Scientist (equity position),"Chicago, IL",55000,Chief Data Scientist*. We are looking for an e...,0,Digital FactoryChief Data Scientist (equity po...
224,Argonne National Laboratory,0,Senior Front End Developer,"Lemont, IL",55000,"The KBase data interfaces (for sharing, displa...",0,Argonne National LaboratorySenior Front End De...
223,Bolstr,0,Chief Credit Officer,"Chicago, IL",55000,Excited to use your experience to build a cred...,0,"BolstrChief Credit OfficerChicago, ILExcited t..."
222,Civis Analytics,0,Software Engineer,"Chicago, IL",55000,Political technology and data. Data visualizat...,0,"Civis AnalyticsSoftware EngineerChicago, ILPol..."
221,Astellas Pharmaceuticals,0,"Clinical Pharmacology Exploratory Development,...","Northbrook, IL",55000, Supervise 1-3 junior CPED scientists.  May ...,0,Astellas PharmaceuticalsClinical Pharmacology ...


In [221]:
deduped_data = data.drop_duplicates(subset='unique_id', keep='last')

In [222]:
deduped_data.count()

Company Name       78
High Salary        78
Job Title          78
Location           78
Salary             78
Summary            78
new_high_salary    78
unique_id          78
dtype: int64

In [224]:
deduped_data['Salary'].describe()

count        78.0
mean     120000.0
std           0.0
min      120000.0
25%      120000.0
50%      120000.0
75%      120000.0
max      120000.0
Name: Salary, dtype: float64

In [225]:
data['unique_id'].value_counts()

AstellasDirector, RWI Data ScienceNorthbrook, ILDevelop data mining and/or data modeling plans to support RWI&A projects and programs. Manage, mentor and develop a high performing data scientists team through...                                                              215
Civis AnalyticsSoftware EngineerChicago, ILPolitical technology and data. Data visualization for the web (using D3 or similar). Engineers collaborate across departments with our data scientists and...                                                                         215
Argonne National LaboratorySenior Front End DeveloperLemont, ILThe KBase data interfaces (for sharing, displaying and accessing biological data),. Experience working with scientific datasets and developing data...                                                            215
BolstrChief Credit OfficerChicago, ILExcited to use your experience to build a credit methodology and decision tree from scratch alongside a wicked smart data scientist.